In [1]:
import json
from glob import glob
from joblib import Parallel, delayed
import pandas as pd

In [2]:
def parse_mediciones_json(json_file):
    "Función auxiliar para convertir los jsons de mediciones de calidad de aire en Pandas Data Frame"
    # Abrimos el archivo json descargado previamente
    with open(json_file, 'r') as aux:
        results = json.load(aux)['results'] # Los resultados vienen en la seccion *results*
    
    # Vamos a guardar los datos de resultados en una lista para después concatenarlos en un DataFrame
    pre_data = []
    
    for r in results:
        # Obtenemos DataFrame previo y lo agregamos
        aux = pd.DataFrame.from_dict(r, orient='index').T
        pre_data.append(aux)
    
    # Concatenamos todos los datos y devolvemos el DataFrame, siempre que haya datos
    if len(pre_data)>0:
        pre_data = pd.concat(pre_data, ignore_index=True)
        return pre_data
    
    # Si por alguna razón no hay datos imprimimos el resultado y no devolvemos nada
    else:
        print('NO DATA FOR ', json_file)
        return

In [7]:
dir_grl = '../data/raw/'
name = dir_grl + 'json/page_%i_sinaica_mediciones.json'

c = 1

for i, j in zip([1, 200], [201, 253]):
    print('Chunk', c)
    aux_list = [name %n for n in range(i, j)]

    sinaica_mediciones = Parallel(n_jobs=-1, verbose=8)(delayed (parse_mediciones_json)(f) for f in aux_list)

    sinaica_mediciones = pd.concat(sinaica_mediciones, ignore_index=True)
    print(sinaica_mediciones.shape)
    sinaica_mediciones.to_csv(dir_grl+'Grl/sinaica_mediciones_%i_%i.csv' %(i, j),
                              index=False)

Chunk 1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 23.9min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 41.1min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 59.2min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 75.2min finished


(4000000, 11)
Chunk 1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed: 20.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  53 out of  53 | elapsed: 20.1min finished


(1060000, 11)


In [9]:
all_data_mediciones = []
for file in glob(dir_grl+'Grl/*.csv'):
    aux = pd.read_csv(file,
                      usecols = ['estacionesid', 'fecha', 'hora', 'city', 'state',
                                 'parametro', 'valororig', 'validoorig'])
    all_data_mediciones.append(aux)
    
all_data_mediciones = pd.concat(all_data_mediciones)
print(all_data_mediciones.shape)
all_data_mediciones.head()

(5060000, 8)


,validoorig,city,state,valororig,parametro,estacionesid,hora,fecha
0,1,Durango,Durango,0.001938,SO2,59,18,2018-07-26
1,1,Durango,Durango,0.001868,SO2,59,19,2018-07-26
2,1,Durango,Durango,0.001779,SO2,59,20,2018-07-26
3,1,Durango,Durango,0.001764,SO2,59,21,2018-07-26
4,1,Durango,Durango,0.001732,SO2,59,22,2018-07-26


In [10]:
all_data_mediciones.to_csv(dir_grl+'Grl/data_mediciones_todas_estaciones.csv', index=False)

In [11]:
estaciones = pd.read_csv(dir_grl+'Grl/stations/estaciones.csv')
# tiramos estaciones fuera del país
mask = (estaciones.lat.between(14, 34.5)) & (estaciones.long.between(-120, -70))
estaciones = estaciones[mask].rename(columns={'id':'estacionesid'})

print(estaciones.shape)
estaciones.head()

(180, 7)


,_id,lat,long,estacionesid,nombre,codigo,redesid
0,5cccf5bee2705c1932820580,21.873311,-102.320803,31,CBTIS,CBT,30
1,5cccf5bee2705c1932820581,21.846392,-102.288431,32,Secretaría de Medio Ambiente,SMA,30
2,5cccf5bee2705c1932820582,21.883781,-102.295825,33,Centro,CEN,30
3,5cccf5bee2705c1932820583,32.639722,-115.506389,39,COBACH,SPABC14,32
4,5cccf5bee2705c1932820584,32.603639,-115.485944,41,CESPM,SPABC19,32


In [12]:
mediciones = pd.read_csv(dir_grl+'Grl/data_mediciones_todas_estaciones.csv')
print(mediciones.shape)
mediciones.head()

(5060000, 8)


,validoorig,city,state,valororig,parametro,estacionesid,hora,fecha
0,1,Durango,Durango,0.001938,SO2,59,18,2018-07-26
1,1,Durango,Durango,0.001868,SO2,59,19,2018-07-26
2,1,Durango,Durango,0.001779,SO2,59,20,2018-07-26
3,1,Durango,Durango,0.001764,SO2,59,21,2018-07-26
4,1,Durango,Durango,0.001732,SO2,59,22,2018-07-26


In [14]:
estaciones_ciudad = mediciones[['city', 'state', 'estacionesid']].drop_duplicates()
print(estaciones_ciudad.shape)
estaciones_ciudad.head()

(135, 3)


,city,state,estacionesid
0,Durango,Durango,59
9502,Durango,Durango,60
56830,Gómez Palacio,Durango,65
75239,Celaya,Guanajuato,68
112849,Celaya,Guanajuato,69


In [21]:
city_stations = pd.merge(estaciones_ciudad, estaciones, left_on='estacionesid',
                        right_on='estacionesid')
city_stations.head(100)

,city,state,estacionesid,_id,lat,long,nombre,codigo,redesid
0,Durango,Durango,59,5cccf5bee2705c1932820592,24.051556,-104.611361,IPN,IPN,42
1,Durango,Durango,60,5cccf5bee2705c1932820593,24.026778,-104.692975,SRNyMA,SRN,42
2,Gómez Palacio,Durango,65,5cccf5bee2705c1932820594,25.551222,-103.502528,Campestre,CAM,43
3,Celaya,Guanajuato,68,5cccf5bee2705c1932820596,20.517500,-100.815556,Policía,POL,45
4,Celaya,Guanajuato,69,5cccf5bee2705c1932820597,20.545278,-100.806944,Tecnológico,TEC,45
...,...,...,...,...,...,...,...,...,...
95,Valle de México,Ciudad de México,268,5cccf5bee2705c1932820606,19.360833,-99.073889,UAM Iztapalapa,UIZ,119
96,Valle de México,Ciudad de México,269,5cccf5bee2705c1932820607,19.304444,-99.103889,UAM Xochimilco,UAX,119
97,Valle de México,Ciudad de México,270,5cccf5bee2705c1932820608,19.657500,-99.096667,Villa de las Flores,VIF,119
98,Valle de México,Ciudad de México,271,5cccf5bee2705c1932820609,19.526111,-99.082500,Xalostoc,XAL,119


In [17]:
city_stations.to_csv(dir_grl+'Grl/stations/city_stations.csv')

In [23]:
city_stations[city_stations['city']=='Valle de México']

,city,state,estacionesid,_id,lat,long,nombre,codigo,redesid
71,Valle de México,Ciudad de México,240,5cccf5bee2705c19328205ed,19.635556,-98.912222,Acolman,ACO,119
72,Valle de México,Ciudad de México,241,5cccf5bee2705c19328205ee,19.154286,-99.162778,Ajusco,AJU,119
73,Valle de México,Ciudad de México,242,5cccf5bee2705c19328205ef,19.272222,-99.207778,Ajusco Medio,AJM,119
74,Valle de México,Ciudad de México,243,5cccf5bee2705c19328205f0,19.577222,-99.254167,Atizapán,ATI,119
75,Valle de México,Ciudad de México,244,5cccf5bee2705c19328205f1,19.468611,-99.170000,Camarones,CAM,119
76,Valle de México,Ciudad de México,245,5cccf5bee2705c19328205f2,19.326389,-99.176111,Centro de Ciencias de la Atmósfera,CCA,119
77,Valle de México,Ciudad de México,246,5cccf5bee2705c19328205f3,19.267222,-98.886111,Chalco,CHO,119
78,Valle de México,Ciudad de México,248,5cccf5bee2705c19328205f5,19.365556,-99.291944,Cuajimalpa,CUA,119
79,Valle de México,Ciudad de México,249,5cccf5bee2705c19328205f6,19.722222,-99.198611,Cuautitlán,CUT,119
80,Valle de México,Ciudad de México,250,5cccf5bee2705c19328205f7,19.482500,-99.243611,FES Acatlán,FAC,119
